In [1]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

f:\UI\Semester 8\Tugas Akhir\topic-modelling\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("GPU available:", torch.cuda.is_available())

GPU available: False


In [ ]:
df = pd.read_csv("../data/stemmed_merged_kubu_02.csv") 
print(df.columns)

Index(['created_at', 'id_str', 'full_text', 'lang', 'location'], dtype='object')


In [4]:
df['full_text'] = df['full_text'].str.replace(r'(wk)+|\bsih\b|\bya\b', '', regex=True)

In [5]:
docs = df['full_text'].dropna().astype(str).tolist()

In [20]:
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
embedding_model.to('cuda' if torch.cuda.is_available() else 'cpu')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [21]:
pca_model = PCA(n_components=25)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=15,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True,
)


In [23]:
topic_model_hdbscan = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    calculate_probabilities=True,
    verbose=True
)

topics_hdbscan, probs = topic_model_hdbscan.fit_transform(docs)

topic_model_hdbscan.reduce_topics(docs, nr_topics=15)


2025-05-21 21:26:16,399 - BERTopic - Embedding - Transforming documents to embeddings.


Batches: 100%|██████████| 934/934 [05:31<00:00,  2.82it/s]
2025-05-21 21:31:48,000 - BERTopic - Embedding - Completed ✓
2025-05-21 21:31:48,002 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-21 21:32:09,055 - BERTopic - Dimensionality - Completed ✓
2025-05-21 21:32:09,057 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-21 21:34:03,614 - BERTopic - Cluster - Completed ✓
2025-05-21 21:34:03,622 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-21 21:34:04,081 - BERTopic - Representation - Completed ✓
2025-05-21 21:34:04,717 - BERTopic - Topic reduction - Reducing number of topics
2025-05-21 21:34:04,743 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-21 21:34:05,040 - BERTopic - Representation - Completed ✓
2025-05-21 21:34:05,045 - BERTopic - Topic reduction - Reduced number of topics from 210 to 15


In [24]:
# Topic summary table
topic_info = topic_model_hdbscan.get_topic_info()
print(topic_info)

# Interactive visualizations
topic_model_hdbscan.visualize_topics()
topic_model_hdbscan.visualize_barchart(top_n_topics=15)
topic_model_hdbscan.visualize_heatmap()


    Topic  Count                                 Name  \
0      -1  10285              -1_01_anies_dukung_imin   
1       0   9630              0_01_dukung_pilih_calon   
2       1   4131      1_anies_presiden_baswedan_pilih   
3       2   1643        2_debat_calon_presiden_kritik   
4       3   1532  3_politik_partai_demokrasi_demokrat   
5       4    917              4_cak_imin_anies_banget   
6       5    641          5_video_videotron_pop_korea   
7       6    446        6_twitter_media_berita_sosial   
8       7    219              7_program_anies_01_misi   
9       8    136      8_palestina_israel_merdeka_aksi   
10      9     98                9_data_fakta_01_pakai   
11     10     61    10_strategi_01_strategy_strategis   
12     11     52          11_respons_reply_respon_cak   
13     12     51          12_viral_viralkan_ham_virus   
14     13     19              13_store_app_unduh_play   

                                       Representation  \
0   [01, anies, dukung, imin, 

In [25]:
topic_info = topic_model_hdbscan.get_topic_info()
display(topic_info.head())        

,Topic,Count,Name,Representation,Representative_Docs
0,-1,10285,-1_01_anies_dukung_imin,"[01, anies, dukung, imin, cak, pilih, calon, a...",[bodoh rakyat malas pilih anies rasyid basweda...
1,0,9630,0_01_dukung_pilih_calon,"[01, dukung, pilih, calon, orang, menang, pasa...","[pilih pasang calon 01 visi misi orang, wajib..."
2,1,4131,1_anies_presiden_baswedan_pilih,"[anies, presiden, baswedan, pilih, imin, cak, ...",[anies anies baswedan gus muhaimin iskandar ba...
3,2,1643,2_debat_calon_presiden_kritik,"[debat, calon, presiden, kritik, cak, imin, wa...","[berani debat calon wakil presiden pasang 01 ,..."
4,3,1532,3_politik_partai_demokrasi_demokrat,"[politik, partai, demokrasi, demokrat, pilih, ...",[kampanye nasional demokrat partai bangkit ban...


In [26]:
for tid in topic_info['Topic'][:15]:   # first 10 topics
    print(f"\nTopic {tid}")
    print(topic_model_hdbscan.get_topic(tid)[:15])  # top-15 words



Topic -1
[('01', np.float64(0.04890438519889406)), ('anies', np.float64(0.03579503385857373)), ('dukung', np.float64(0.03236362162071922)), ('imin', np.float64(0.02788732722813066)), ('cak', np.float64(0.02755967197237106)), ('pilih', np.float64(0.022611938136390872)), ('calon', np.float64(0.019170190770975096)), ('amin', np.float64(0.019057496947465263)), ('orang', np.float64(0.018737337311561048)), ('pasang', np.float64(0.0176480992051348))]

Topic 0
[('01', np.float64(0.07773952590163129)), ('dukung', np.float64(0.03791428150165297)), ('pilih', np.float64(0.029938787591593564)), ('calon', np.float64(0.028133333415361585)), ('orang', np.float64(0.024996824995156874)), ('menang', np.float64(0.02490594450086572)), ('pasang', np.float64(0.023301238832070498)), ('kayak', np.float64(0.021378630836556858)), ('banget', np.float64(0.01775913753213697)), ('presiden', np.float64(0.016860084702665704))]

Topic 1
[('anies', np.float64(0.07414402235701806)), ('presiden', np.float64(0.05759720944

In [27]:
topic_model_hdbscan.visualize_barchart(top_n_topics=15)
topic_model_hdbscan.visualize_topics()

In [ ]:
# df['created_at'] = pd.to_datetime(df['created_at'])

In [ ]:
# topics_over_time = topic_model_hdbscan.topics_over_time(
#     docs=docs,
#     topics=topics_hdbscan,
#     timestamps=df['created_at'],
#     nr_bins=20,  # Adjust to control time granularity
#     evolution_tuning=True,
#     global_tuning=True
# )

0it [00:00, ?it/s]


IndexError: index (206) out of range

In [ ]:
# topic_model_hdbscan.visualize_topics_over_time(
#     topics_over_time, 
#     top_n_topics=10  # Number of top topics to show
# )


In [ ]:
# Save results to CSV
df_topics = pd.DataFrame({"Document": docs, "Topic": topics_hdbscan})
df_topics.to_csv("topic_assignments_hdbscan.csv", index=False)

# Save model
# topic_model.save("bertopic_model_hdbscan")


In [ ]:
from sklearn.cluster import KMeans

num_topics = 15
kmeans_model = KMeans(n_clusters=num_topics, random_state=42)

topic_model_kmeans = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=kmeans_model,
    calculate_probabilities=False,
    verbose=True
)

topics_kmeans, _ = topic_model_kmeans.fit_transform(docs)

topic_model_kmeans.visualize_barchart(top_n_topics=15)

2025-05-20 19:36:19,776 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 934/934 [04:49<00:00,  3.23it/s]
2025-05-20 19:41:09,173 - BERTopic - Embedding - Completed ✓
2025-05-20 19:41:09,174 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-20 19:41:14,917 - BERTopic - Dimensionality - Completed ✓
2025-05-20 19:41:14,920 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-20 19:41:14,978 - BERTopic - Cluster - Completed ✓
2025-05-20 19:41:14,986 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-20 19:41:15,365 - BERTopic - Representation - Completed ✓


In [ ]:
# Topic summary table
topic_info = topic_model_kmeans.get_topic_info()
print(topic_info)

# Interactive visualizations
topic_model_kmeans.visualize_topics()
topic_model_kmeans.visualize_barchart(top_n_topics=15)
topic_model_kmeans.visualize_heatmap()


    Topic  Count                            Name  \
0       0   3525          0_01_dukung_pilih_amin   
1       1   3463         1_01_dukung_orang_pilih   
2       2   3304        2_01_dukung_pasang_pilih   
3       3   2851       3_anies_baswedan_imin_cak   
4       4   2314    4_presiden_anies_pilih_calon   
5       5   2271      5_01_kampanye_dukung_kerja   
6       6   2269      6_anies_indonesia_imin_cak   
7       7   2033         7_cak_imin_anies_banget   
8       8   1730      8_kayak_suara_01_indonesia   
9       9   1416       9_presiden_calon_wakil_01   
10     10   1286        10_menang_01_survei_gara   
11     11   1056     11_video_program_01_twitter   
12     12    871  12_politik_demokrasi_partai_01   
13     13    762      13_debat_calon_01_presiden   
14     14    710      14_debat_cak_imin_presiden   

                                       Representation  \
0   [01, dukung, pilih, amin, buzzer, coblos, kubu...   
1   [01, dukung, orang, pilih, pasang, calon, bang...

In [ ]:
topic_info = topic_model_kmeans.get_topic_info()
display(topic_info.head())        

,Topic,Count,Name,Representation,Representative_Docs
0,0,3525,0_01_dukung_pilih_amin,"[01, dukung, pilih, amin, buzzer, coblos, kubu...","[lucu dukung 01, pas pilih amin 01, coblos 01 ..."
1,1,3463,1_01_dukung_orang_pilih,"[01, dukung, orang, pilih, pasang, calon, bang...",[wajib alas pilih pasang calon deh dukung pasa...
2,2,3304,2_01_dukung_pasang_pilih,"[01, dukung, pasang, pilih, banget, buzzer, ca...","[dukung 01 banyak korban buzzer deh, komen duk..."
3,3,2851,3_anies_baswedan_imin_cak,"[anies, baswedan, imin, cak, rasyid, ayah, ami...",[anak sebut lengkap anies rasyid baswedan tema...
4,4,2314,4_presiden_anies_pilih_calon,"[presiden, anies, pilih, calon, baswedan, waki...",[cak imin sebut santri calon presiden calon wa...


In [ ]:
for tid in topic_info['Topic'][:15]:   # first 10 topics
    print(f"\nTopic {tid}")
    print(topic_model_kmeans.get_topic(tid)[:15])  # top-15 words



Topic 0
[('01', np.float64(0.17999234753117513)), ('dukung', np.float64(0.05316047423135683)), ('pilih', np.float64(0.038002265207892454)), ('amin', np.float64(0.02456071220150472)), ('buzzer', np.float64(0.02219320211689753)), ('coblos', np.float64(0.02117551507583413)), ('kubu', np.float64(0.0188911609046937)), ('bang', np.float64(0.018724551401291385)), ('banget', np.float64(0.01695381068698922)), ('all', np.float64(0.015265770499369108))]

Topic 1
[('01', np.float64(0.06116502636690249)), ('dukung', np.float64(0.042540085387303604)), ('orang', np.float64(0.04154855264352592)), ('pilih', np.float64(0.03055361364644349)), ('pasang', np.float64(0.028660519732379448)), ('calon', np.float64(0.0212359306588387)), ('banget', np.float64(0.019680014415701514)), ('teman', np.float64(0.019485637074210575)), ('bilang', np.float64(0.01640976637701461)), ('anies', np.float64(0.014695961789340933))]

Topic 2
[('01', np.float64(0.1029656144038641)), ('dukung', np.float64(0.06065905724951994)), ('

In [ ]:
topic_model_kmeans.visualize_barchart(top_n_topics=15)
topic_model_kmeans.visualize_topics()

In [ ]:
# Save results to CSV
df_topics = pd.DataFrame({"Document": docs, "Topic": topics_kmeans})
df_topics.to_csv("topic_assignments_kmeans.csv", index=False)

# Save model
# topic_model.save("bertopic_model_kmeans")

In [ ]:
topics_over_time_kmeans = topic_model_kmeans.topics_over_time(
    docs=docs,
    topics=topics_hdbscan,
    timestamps=df['created_at'],
    nr_bins=20,
    evolution_tuning=True,
    global_tuning=True
)

topic_model_hdbscan.visualize_topics_over_time(topics_over_time_kmeans, top_n_topics=10)

In [ ]:
print("HDBSCAN Topics:", len(set(topics_hdbscan)) - (1 if -1 in topics_hdbscan else 0))
print("KMeans Topics:", len(set(topics_kmeans)))

# Outlier analysis
print("HDBSCAN Outliers:", topics_hdbscan.count(-1))  # KMeans won't have outliers

HDBSCAN Topics: 205
KMeans Topics: 15
HDBSCAN Outliers: 9996


In [ ]:
import pandas as pd
from collections import Counter

# Count documents per topic
hdbscan_counts = Counter(topics_hdbscan)
kmeans_counts = Counter(topics_kmeans)

# Create DataFrame to compare
comparison_df = pd.DataFrame({
    "HDBSCAN": pd.Series(hdbscan_counts),
    "KMeans": pd.Series(kmeans_counts)
}).fillna(0).astype(int)

comparison_df.sort_index()


,HDBSCAN,KMeans
-1,9996,0
0,941,3525
1,903,3463
2,792,3304
3,789,2851
...,...,...
200,16,0
201,16,0
202,15,0
203,15,0
